<h1><b> Set-up to Import the Spotify Dataset to MySQL Workbench! </b></h1>

<H3><b>Install Necessary Libraries</b></H3>

In [2]:
pip install mysql-connector-python pandas gdown

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymysql

   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB 640.0 kB/s eta 0:00:01
   --------------------------- ------------ 30.7/45.0 kB 640.0 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 245.8 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 245.8 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 245.8 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 147.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


<H3><b>Import Data</b></H3>

In [21]:
# Step 1: Import necessary libraries
import gdown
import pandas as pd
import pymysql
from pymysql import OperationalError

In [23]:
# Step 2: Download the dataset from Google Drive
file_id = '1009ZQdqIQV1-TNqNt1rXENK4zFWM_55u'
output = 'spotify_dataset.csv'
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1009ZQdqIQV1-TNqNt1rXENK4zFWM_55u
To: C:\Users\Richard Muchoki\Documents\SQL\Spotify_Dataset_Analysis\spotify_dataset.csv
100%|██████████| 21.3M/21.3M [01:03<00:00, 337kB/s]


'spotify_dataset.csv'

<p style="font-size:16px;"><b>Note:</b><br>
    
    ** From the shareable link, extract the file ID. The file ID is the part after /d/ and before /view. **

<b>For example, if your link is:</b>

<spam>https://drive.google.com/file/d/1009ZQdqIQV1-TNqNt1rXENK4zFWM_55u/view?usp=drive_link</spam>

The file ID is <b>1009ZQdqIQV1-TNqNt1rXENK4zFWM_55u.</b>
</p>

In [25]:
# Step 3: Load the dataset into a pandas DataFrame
df = pd.read_csv(output)

In [27]:
# Display the first few rows of the DataFrame to confirm successful loading
print(df.head())

        spotify_track_uri                   ts    platform  ms_played  \
0  2J3n32GeLmMjwuAzyhcSNe  2013-07-08 02:44:34  web player       3185   
1  1oHxIPqJyvAYHy0PVrDU98  2013-07-08 02:45:37  web player      61865   
2  487OPlneJNni3NWC8SYqhW  2013-07-08 02:50:24  web player     285386   
3  5IyblF777jLZj1vGHG2UD3  2013-07-08 02:52:40  web player     134022   
4  0GgAAB0ZMllFhbNc3mAodO  2013-07-08 03:17:52  web player          0   

                                      track_name        artist_name  \
0                            Say It, Just Say It       The Mowgli's   
1  Drinking from the Bottle (feat. Tinie Tempah)      Calvin Harris   
2                                    Born To Die       Lana Del Rey   
3                               Off To The Races       Lana Del Rey   
4                                      Half Mast  Empire Of The Sun   

                           album_name reason_start reason_end  shuffle  \
0                Waiting For The Dawn     autoplay   clickro

In [31]:
# Check the number of rows and columns respectively
# Initial output: 149,860 rows and 11 columns
df.shape

(149860, 11)

In [33]:
# Step 4: Handle NaN values by replacing them with appropriate defaults
df.fillna({
    'spotify_track_uri': '',
    'ts': '1970-01-01 00:00:00',
    'platform': '',
    'ms_played': 0,
    'track_name': '',
    'artist_name': '',
    'album_name': '',
    'reason_start': '',
    'reason_end': '',
    'shuffle': False,
    'skipped': False
}, inplace=True)

In [35]:
# Check for duplicates and print them for the unique identifiers of our data
duplicates = df[df.duplicated(subset=['spotify_track_uri', 'ts'], keep=False)]
print(duplicates)

             spotify_track_uri                   ts platform  ms_played  \
1179    3U4isOIWM3VvDubwSI3y7a  2015-08-29 16:45:52  android       3042   
1180    3U4isOIWM3VvDubwSI3y7a  2015-08-29 16:45:52  android       4179   
1533    6y63IkVtjr2RnPVvK8BqEj  2015-09-01 05:05:18  windows        333   
1535    6y63IkVtjr2RnPVvK8BqEj  2015-09-01 05:05:18  android       1531   
1659    3fWNnzF9L1z5m8LJ3UJREV  2015-09-10 02:37:21  android       2547   
...                        ...                  ...      ...        ...   
125713  4401c08DdNwwGEg8WGCkQf  2022-10-01 23:18:48  android       1611   
125760  0LxCY6cjKgjutOZaqyjrVQ  2022-10-02 18:34:27  android     221693   
125761  0LxCY6cjKgjutOZaqyjrVQ  2022-10-02 18:34:27  android     221693   
126078  21s2FWRApO7LxvhunUoPNm  2022-10-10 06:00:48  android       1794   
126079  21s2FWRApO7LxvhunUoPNm  2022-10-10 06:00:48  android     186549   

                                track_name  artist_name  \
1179                             All of 

In [37]:
# let's measure the extent of duplicates
# Calculate the percentage of exact duplicates
total_records = df.shape[0]
duplicate_records = duplicates.shape[0]
percentage_duplicates = (duplicate_records / total_records) * 100
print(f"Percentage of duplicates: {percentage_duplicates:.2f}%")

Percentage of duplicates: 1.92%


In [43]:
# Analyze the Impact of Duplicates
# Compare statistics with and without duplicates
print("Statistics with duplicates:")
print(df.describe())

# Remove exact duplicates
df_no_duplicates = df.drop_duplicates()

print("Statistics without duplicates:")
print(df_no_duplicates.describe())

Statistics with duplicates:
          ms_played
count  1.498600e+05
mean   1.283166e+05
std    1.178401e+05
min    0.000000e+00
25%    2.795000e+03
50%    1.388400e+05
75%    2.185070e+05
max    1.561125e+06
Statistics without duplicates:
          ms_played
count  1.486750e+05
mean   1.280607e+05
std    1.178178e+05
min    0.000000e+00
25%    2.786000e+03
50%    1.383860e+05
75%    2.183330e+05
max    1.561125e+06


<p style="font-size:16px;"><b> The duplicates have no significant effect on the statistics hence we can safely drop them </b></p>

In [47]:
# Step 5: Remove duplicate entries based on the primary key
df.drop_duplicates(subset=['spotify_track_uri', 'ts'], inplace=True)

In [51]:
# Step 6: Convert DataFrame to list of tuples for bulk insert
data = df.to_records(index=False).tolist()

In [53]:
# Step 7: Connect to the MySQL database
try:
    connection = pymysql.connect(
        host='localhost',      # Replace with your MySQL host
        user='root',  # Replace with your MySQL username
        password='12345678',  # Replace with your MySQL password
        database='spotify_analysis'  # Replace with your MySQL database name
    )
    if connection.open:
        print("Successfully connected to the database")
        
        # Step 8: Create a cursor object
        cursor = connection.cursor()

        # Step 9: Create a table to store the dataset
        create_table_query = """
        CREATE TABLE IF NOT EXISTS spotify_data (
            spotify_track_uri VARCHAR(255),
            ts TIMESTAMP,
            platform VARCHAR(255),
            ms_played INT,
            track_name VARCHAR(255),
            artist_name VARCHAR(255),
            album_name VARCHAR(255),
            reason_start VARCHAR(255),
            reason_end VARCHAR(255),
            shuffle BOOLEAN,
            skipped BOOLEAN,
            PRIMARY KEY (spotify_track_uri, ts)
        )
        """
        cursor.execute(create_table_query)
        print("Table created successfully")

         # Step 10: Define the bulk insert query
        insert_query = """
        INSERT IGNORE INTO spotify_data (
            spotify_track_uri, ts, platform, ms_played, track_name, artist_name, album_name, reason_start, reason_end, shuffle, skipped
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Step 11: Perform bulk insert
        cursor.executemany(insert_query, data)

        # Commit the transaction
        connection.commit()
        print("Data inserted successfully")


except OperationalError as e:
    print(f"Error: {e}")
finally:
    if connection and connection.open:
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Successfully connected to the database
Table created successfully
Data inserted successfully
MySQL connection is closed
